# Imports 

In [1]:
import numpy as np

# Task 3

Learning by doing: the best way to understand the basics of quantum computation is to implement a quantum circuit simulator. This task is suitable both for people from computer sciences who want to learn about quantum computing, and for people from math/physics who want to exercise coding.

Detailed description of the task with some learning resources and examples can be found in this [jupyter notebook](https://github.com/quantastica/qosf-mentorship/blob/master/qosf-simulator-task.ipynb)

It is expected that simulator can perform following:

-initialize state

-read program, and for each gate:
    
    -calculate matrix operator
    
    -apply operator (modify state)

-perform multi-shot measurement of all qubits using weighted random technique

# Criteria 

### Coding skills – clear, readable, well-structured code

### Communication – well-described results, easy to understand, tidy.

### Reliability – submitted on time, all the points from the task description are met

### Research skills – asking good questions and answering them methodically



# Here are some things we want to accomplish as a starting point for a QC compiler 

In [2]:
def get_ground_state(num_qubits):
    # return vector of size 2**num_qubits with all zeroes except first element which is 1
    return

def get_operator(total_qubits, gate_unitary, target_qubits):
    # return unitary operator of size 2**n x 2**n for given gate and target qubits
    return

def run_program(initial_state, program):
    # read program, and for each gate:
    #   - calculate matrix operator
    #   - multiply state with operator
    # return final statevb
    return

def measure_all(state_vector):
    # choose element from state_vector using weighted random and return it's index
    return

def get_counts(state_vector, num_shots):
    # simply execute measure_all in a loop num_shots times and
    # return object with statistics in following form:
    #   {
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      ...
    #   }
    # (only for elements which occoured - returned from measure_all)
    return

In [3]:
def get_ground_state(num_qubits):
    
    ''' 
    return vector of size 2**num_qubits with all zeroes except first element which is 1, 
    this vector represents all qubits in the system being in the |0> state. 
    
    example: 
    
        1 qubit - |10>        Two possible states
        2 qubit - |1000>      Four possible states
        3 qubit - |100000000> Eight possible states
        
    '''
    
    import numpy as np 
    
    number_of_states = 2**num_qubits
    
    # I use np.pad() to add some number of zeros to the end of an array , this is better than a loop
    # which is the obvious solution but not the best solution. 
    
    ground_state = np.array([1])
    
    ground_state = np.pad(ground_state, (0, number_of_states-1), 'constant')
    
    return ground_state
    

In [4]:
gs=get_ground_state(5)
print(' Ground State: ', gs, '\n', 'Number of possible states: ', len(gs))


 Ground State:  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 
 Number of possible states:  32


# Fundamental Gates

X, Y, Z, H, I, CNOT


In [5]:
# These single qubit gates are easy enough to hard code in 

X = np.array([
[0, 1],
[1, 0]
])

Y = np.array([
[0, -1j],
[1j, 0]
])

Z = np.array([
[1, 0],
[0, -1]
])

H = np.array([
[1/np.sqrt(2), 1/np.sqrt(2)],
[1/np.sqrt(2), -1/np.sqrt(2)]
])

I=np.identity(2)

# Creating a mapping of strings to gates 

gates={'X':X, 'Y':Y, 'Z':Z, 'H':H, 'I': I}

# Single Qubit Operations with  our fundamental 2x2 gates

In [6]:
initial_state = get_ground_state(1)
print('Initial State: ', initial_state)


gate='H'

print( gate, ' on ', initial_state)

final_state = np.dot(gates[gate], initial_state)
print('Final State: ', final_state)




Initial State:  [1 0]
H  on  [1 0]
Final State:  [0.70710678 0.70710678]


## We can wrap this in a function to check all our single qubit operations easily 

In [7]:
def single_qubit_op(gate, initial_state): 
    
    '''
    
    Take a gate from the dictionary of single qubit gates, and apply it to an initial state
    
    example:
    
        X on |10> will yield |01> 
        X on |01> will yield |10> 
        
    
    '''
    
    import numpy as np 

    # These single qubit gates are easy enough to hard code in 

    X = np.array([
    [0, 1],
    [1, 0]
    ])

    Y = np.array([
    [0, -1j],
    [1j, 0]
    ])

    Z = np.array([
    [1, 0],
    [0, -1]
    ])

    H = np.array([
    [1/np.sqrt(2), 1/np.sqrt(2)],
    [1/np.sqrt(2), -1/np.sqrt(2)]
    ])

    I=np.identity(2) 

    # Creating a mapping of strings to gates 

    gates={'X':X, 'Y':Y, 'Z':Z, 'H':H, 'I': I}
    
    
    final_state = np.dot(gates[gate], initial_state)
    
    
    return final_state

## Our expected results from using the X gate on some initial states 

In [8]:
print('X[0,1] = ' , single_qubit_op('X', [0,1]))

print('X[1,0] = ' , single_qubit_op('X', [1,0])) 

X[0,1] =  [1 0]
X[1,0] =  [0 1]


### Now that we have a place to start we need to generalize to multiple qubit systems



# Matrix operator

Quantum program eveloves quantum state by multiplying state vector with each gate's unitary matrix (dot product). Note that dimension of the state vector and dimension of the unitary matrix describing a gate usually don't match. For example: 3-qubit quantum circuit's state vector has $2^n=2^3=8$ elements, but single-qubit gate has $2^n\times2^n=2^1\times2^1=2\times2$ elements. In order to perform matrix-vector multiplication, we need to "resize" gate's matrix to the dimension of the state vector. Let's call that matrix a matrix operator.

Note that size of the matrix operator is $2^n\times2^n$ where $n$ is total number of qubits in the circuit, so storing it into memory and calculating it requires a lot of memory and cpu power for bigger circuits. Optimizing this code is most interesting and challenging part, but for our purpose it is enough if you make it work smoothly with 8 qubits (the more - the better).

Matrix operator for single-qubit gates
Matrix operator for single-qubit gate can be calculated by performing tensor product of gate's unitary matrix and $2\times2$ identity matrices in correct order.

Example for single-qubit gate $U$ in 3-qubit circuit:

gate on qubit 0: ${O=U \otimes I \otimes I}$
gate on qubit 1: ${O=I \otimes U \otimes I}$
gate on qubit 2: ${O=I \otimes I \otimes U}$
Example matrix operator for X gate acting on third qubit in 3-qubit circuit can be calculated like this:


In [9]:
# Supplement code as a starting point 

In [10]:
import numpy as np

# Let's define state vector of the 3-qubit circuit in "ground state" (all qubits in state |0>)

psi = [1, 0, 0, 0, 0, 0, 0, 0] 
print("Initial state:", psi) 


# Define X (NOT) gate: 

X = np.array([
[0, 1],
[1, 0]
])

# Define 2x2 identity

I = np.identity(2)

# Calculate operator for X gate acting on third qubit in 3-qubit circuit 

O = np.kron(np.kron(I, I), X) 

print("\nOperator:\n\n", O, "\n")

# And finally, apply operator

psi = np.dot(psi, O)
print("Final state:", psi) 

Initial state: [1, 0, 0, 0, 0, 0, 0, 0]

Operator:

 [[0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]] 

Final state: [0. 1. 0. 0. 0. 0. 0. 0.]


# Single qubit operations for systems of multiple qubits 

### generalized we get 

where n is the total number of qubits in our system 

On 0th qubit: ${O=U \otimes I^{n-1}}$

On 1st qubit: ${O=I \otimes U \otimes I^{n-2}}$

On 2nd qubit: ${O=I \otimes I\otimes U \otimes I^{n-3}}$

On 3rd qubit: ${O=I \otimes I \otimes I\otimes U \otimes I^{n-4}}$

...

On mth qubit: ${O=I^{m} \otimes U \otimes I^{n-m-1)}}$

we can code this out and be able to use a single qubit operation on some mth qubit in a n-qubit arrangement 
like the supplemental material I will try it out with the X gate first

At first thought I think there is going to be opprotunity to use recursion to calculate the matrices instead doing some big for loops for larger quantum circuits this will save us cpu and memory that we desperately need 


I realize that big endian would be reversed....

In [11]:
# intermediate version of get_operator() that only includes single qubit operations 

def make_operator(total_qubits, target_qubit, unitary_operator):
    
    '''
    This works with forloops and is for big endian encoding , the least significant qubit , the zeroth qubit is to the left,
    the most significant to the right. 
    
    where n is the total number of qubits in our system, in big endian the operators are simply reversed  

    On 0th qubit: ${O=U \otimes I^{n-1}}$

    On 1st qubit: ${O=I \otimes U \otimes I^{n-2}}$

    On 2nd qubit: ${O=I \otimes I\otimes U \otimes I^{n-3}}$

    On 3rd qubit: ${O=I \otimes I \otimes I\otimes U \otimes I^{n-4}}$

    ...

    On mth qubit: ${O=I^{m} \otimes U \otimes I^{n-m-1)}}$
    
    
    '''
    
    import numpy as np

    O=1
    prev=I
    U=unitary_operator
    string=''

    for i in range(total_qubits-target_qubit): 
        O=np.kron(O,I)
        string+='I'

    O=np.kron(O,U)
    string+='U'

    for i in range(target_qubit-1): 
        O=np.kron(O,I)
        string+='I'

    print('O: ', string)
    return O 

Now we need to have our make_operator() work for CNOT which is a multiple qubit gate, it takes a target and control qubit




In [12]:
# Define |0><0| and |1><1| projection operators to help us write out CNOT 

P0=np.array([
    [1,0],
    [0,0]
])

P1=np.array([
    [0,0],
    [0,1]
])

In [13]:
# # now we have a set up to try our CNOT gate 

# #order matters for contol bits, the operations are reversed if the order of control and target bits are swapped 
# # CX12 != CX21

# # if forward 1 -> 2

# if control_qubit < target_qubit:

#     Operator_0 = np.kron(P0, np.identity(2**(total_qubits-1))) # always will be Identity matrices 

#     Operator_1 =np.kron( np.kron( np.kron( P1, np.identity(2**(target_qubit-control_qubit-1))) , U),
#                             np.identity(2**(total_qubits-target_qubit)))




#     O = Operator_0 + Operator_1



# # if backward 2 -> 1

# if target_qubit < control_qubit: 

#     Operator_0 = np.kron(np.identity(2**(total_qubits-1)), P0) # always wi ll be Identity matrices 

#     Operator_1 =np.kron( 
#                         np.kron( 
#                         np.kron(
#                                 np.identity(2**(total_qubits-target_qubit)), U ) ,
#                          np.identity(2**(target_qubit-control_qubit-1))),P1 )

#     O = Operator_0 + Operator_1



In [14]:
def get_operator(total_qubits, gate_unitary, target_qubits):
    # return unitary operator of size 2**n x 2**n for given gate and target qubits
    
    '''
    This works with forloops and is for big endian encoding , the least significant qubit , the zeroth qubit is to the left,
    the most significant to the right. 
    
    where n is the total number of qubits in our system, in big endian the operators are simply reversed  

    On 0th qubit: ${O=U \otimes I^{n-1}}$

    On 1st qubit: ${O=I \otimes U \otimes I^{n-2}}$

    On 2nd qubit: ${O=I \otimes I\otimes U \otimes I^{n-3}}$

    On 3rd qubit: ${O=I \otimes I \otimes I\otimes U \otimes I^{n-4}}$

    ...

    On mth qubit: ${O=I^{m} \otimes U \otimes I^{n-m-1)}}$
 
        
    '''
    import numpy as np 

    # These single qubit gates are easy enough to hard code in 

    X = np.array([
    [0, 1],
    [1, 0]
    ])

    Y = np.array([
    [0, -1j],
    [1j, 0]
    ])

    Z = np.array([
    [1, 0],
    [0, -1]
    ])

    H = np.array([
    [1/np.sqrt(2), 1/np.sqrt(2)],
    [1/np.sqrt(2), -1/np.sqrt(2)]
    ])

    I=np.identity(2) 

    P0=np.array([
    [1,0],
    [0,0]
    ])

    P1=np.array([
    [0,0],
    [0,1]
    ])
    
    
    gates={'X':X, 'Y':Y, 'Z':Z, 'H':H, 'I': I}
    
    
    #order matters for contol bits, the operations are reversed if the order of control and target bits are swapped 
    # CX12 != CX21
    
    # case if we have a control gate 
    # target_qubits need two inputs here not 1, a control qubit first and a target qubit second
    if 'CX' in gate_unitary:
        control_qubit=target_qubits[0]
        target_qubit=target_qubits[1]
        
        U=X
        
        if control_qubit < target_qubit:

            Operator_0 = np.kron(np.identity(2**(total_qubits-1)), P0) # always will be Identity matrices
            
            
            Operator_1 =np.kron( 
                                np.kron( 
                                np.kron(
                                        np.identity(2**(total_qubits-target_qubit)), U ) ,
                                 np.identity(2**(target_qubit-control_qubit-1))),P1 )

            
            O = Operator_0 + Operator_1
        elif target_qubit < control_qubit: 
            Operator_0 = np.kron(P0, np.identity(2**(total_qubits-1))) # always will be Identity matrices 


            Operator_1 =np.kron( np.kron( np.kron(P1, np.identity(2**(control_qubit-target_qubit-1))) , U),
                                    np.identity(2**(total_qubits-control_qubit)))
            

            O = Operator_0 + Operator_1
   
        
        
        
        
    else: 
        # we have a single qubit
    
        O=1
        U=gates[gate_unitary]
        target_qubit=target_qubits[0]
        for i in range(total_qubits-target_qubit): 
            O=np.kron(O,I)

        O=np.kron(O,U)

        for i in range(target_qubit-1): 
            O=np.kron(O,I)


    
    Operator=O
    
    return Operator





In [15]:
# making sure cnot works! for the backwards case especially :) 
CX=get_operator(2,'CX', [1,2])
gs=get_ground_state(2)

nX=get_operator(2,'X',[1])

initial_state=np.dot(gs, nX)
print(initial_state)
np.dot(initial_state, CX)


[0. 1. 0. 0.]


array([0., 0., 0., 1.])

In [16]:

def run_program(initial_state, program):
    # read program, and for each gate:
    #   - calculate matrix operator
    #   - multiply state with operator
    # return final state
    
    #     my_circuit = [
    # { "gate": "h", "target": [0] }, 
    # { "gate": "cx", "target": [0, 1] }
    # ]
    

    # have program in a specific form a list works out as provided 
    
    
    from math import log2 
    
    psi=initial_state
    total_qubits=int(log2(len(initial_state)))
    for i in range(len(program)): 
        
        gate_unitary=program[i]['gate']
        
        target_qubits=program[i]['target']
        
        O = get_operator(total_qubits, gate_unitary, target_qubits)
        
        psi=np.dot(psi,O)
        
    
    return psi

### Check if run_program() works! 

In [17]:
    my_circuit = [
{ "gate": "H", "target": [1] },{ "gate": "CX", "target": [1, 2] }]

initial_state=get_ground_state(2)
run_program(initial_state, my_circuit)

array([0.70710678, 0.        , 0.        , 0.70710678])

In [18]:
print('initial state: ', initial_state)
prev=np.dot(initial_state,get_operator(2,'H',[1]))
print('After Hadammard: ', prev)
final_state = np.dot(prev,get_operator(2,'CX',[1,2]))
print('final state: ', final_state)

initial state:  [1 0 0 0]
After Hadammard:  [0.70710678 0.70710678 0.         0.        ]
final state:  [0.70710678 0.         0.         0.70710678]


### Now we need to explore measure_all() as we fight towards the end 

In [19]:

def measure_all(state_vector):
    # choose element from state_vector using weighted random and return it's index
    
    from math import log2 
    import numpy as np
    

    total_qubits=int(log2(len(state_vector)))
        
    # generate a list of possible states 
    
    possible_states=[]
    
    for i in range(len(state_vector)):
    
        string=bin(i)[2:][::-1] # helps get binary representation of states in our encoding of big endian 
                                #first splice gets rid of the '0b' in front of the string returned by bin
                                # second splice reverses the string 
        
        string+='0'*(total_qubits-len(string))
        
        possible_states.append(string)
    
    probability_distribution=(np.array(state_vector))**2
    result=np.random.choice(possible_states,1, p=probability_distribution)
    
    return result[0]

# testing measure_all()

In [20]:
print('initial state: ', initial_state)
prev=np.dot(initial_state,get_operator(2,'H',[1]))
print('After H: ', prev)

result=measure_all(prev)
print('result: ', result)

initial state:  [1 0 0 0]
After H:  [0.70710678 0.70710678 0.         0.        ]
result:  10


# get_counts() is measuring just a bunch of times and keeping record of our results!

In [21]:
def get_counts(state_vector, num_shots):
    # simply execute measure_all in a loop num_shots times and
    # return object with statistics in following form:
    #   {
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      ...
    #   }
    # (only for elements which occoured - returned from measure_all)
    
    counts={}
    
    for i in range(num_shots): 
        
        result=measure_all(state_vector)
        
        if result in counts: 
            counts[result]+=1
        else:
            counts[result]=1
    return counts

### Try get_counts()

In [22]:
print('initial state: ', initial_state)
prev=np.dot(initial_state,get_operator(2,'H',[1]))
print('After H: ', prev)

print('results after 1000 measurements: ', get_counts(prev, 1000))

initial state:  [1 0 0 0]
After H:  [0.70710678 0.70710678 0.         0.        ]
results after 1000 measurements:  {'00': 492, '10': 508}


# Task 3 completed! 

In [23]:
def get_ground_state(num_qubits):
    
    ''' 
    return vector of size 2**num_qubits with all zeroes except first element which is 1, 
    this vector represents all qubits in the system being in the |0> state. 
    
    example: 
    
        1 qubit - |10>        Two possible states
        2 qubit - |1000>      Four possible states
        3 qubit - |100000000> Eight possible states
        
    '''
    
    import numpy as np 
    
    number_of_states = 2**num_qubits
    
    # I use np.pad() to add some number of zeros to the end of an array , this is better than a loop
    # which is the obvious solution but not the best solution. 
    
    ground_state = np.array([1])
    
    ground_state = np.pad(ground_state, (0, number_of_states-1), 'constant')
    
    return ground_state
    

In [24]:
def get_operator(total_qubits, gate_unitary, target_qubits):
    # return unitary operator of size 2**n x 2**n for given gate and target qubits
    
    '''
    This works with forloops and is for big endian encoding , the least significant qubit , the zeroth qubit is to the left,
    the most significant to the right. 
    
    where n is the total number of qubits in our system, in big endian the operators are simply reversed  

    On 0th qubit: ${O=U \otimes I^{n-1}}$

    On 1st qubit: ${O=I \otimes U \otimes I^{n-2}}$

    On 2nd qubit: ${O=I \otimes I\otimes U \otimes I^{n-3}}$

    On 3rd qubit: ${O=I \otimes I \otimes I\otimes U \otimes I^{n-4}}$

    ...

    On mth qubit: ${O=I^{m} \otimes U \otimes I^{n-m-1)}}$
 
        
    '''
    import numpy as np 

    # These single qubit gates are easy enough to hard code in 

    X = np.array([
    [0, 1],
    [1, 0]
    ])

    Y = np.array([
    [0, -1j],
    [1j, 0]
    ])

    Z = np.array([
    [1, 0],
    [0, -1]
    ])

    H = np.array([
    [1/np.sqrt(2), 1/np.sqrt(2)],
    [1/np.sqrt(2), -1/np.sqrt(2)]
    ])

    I=np.identity(2) 

    P0=np.array([
    [1,0],
    [0,0]
    ])

    P1=np.array([
    [0,0],
    [0,1]
    ])
    
    gates={'X':X, 'Y':Y, 'Z':Z, 'H':H, 'I': I}
    
    
    #order matters for contol bits, the operations are reversed if the order of control and target bits are swapped 
    # CX12 != CX21
    
    # case if we have a control gate 
    # target_qubits need two inputs here not 1, a control qubit first and a target qubit second
    if 'CX' in gate_unitary:
        control_qubit=target_qubits[0]
        target_qubit=target_qubits[1]
        
        U=X
        
        if control_qubit < target_qubit:

            Operator_0 = np.kron(np.identity(2**(total_qubits-1)), P0) # always will be Identity matrices
            
            
            Operator_1 =np.kron( 
                                np.kron( 
                                np.kron(
                                        np.identity(2**(total_qubits-target_qubit)), U ) ,
                                 np.identity(2**(target_qubit-control_qubit-1))),P1 )

            
            O = Operator_0 + Operator_1
        elif target_qubit < control_qubit: 
            Operator_0 = np.kron(P0, np.identity(2**(total_qubits-1))) # always will be Identity matrices 


            Operator_1 =np.kron( np.kron( np.kron(P1, np.identity(2**(control_qubit-target_qubit-1))) , U),
                                    np.identity(2**(total_qubits-control_qubit)))
            

            O = Operator_0 + Operator_1
      
    else: 
        # we have a single qubit
    
        O=1
        U=gates[gate_unitary]
        target_qubit=target_qubits[0]
        for i in range(total_qubits-target_qubit): 
            O=np.kron(O,I)

        O=np.kron(O,U)

        for i in range(target_qubit-1): 
            O=np.kron(O,I)
  
    Operator=O
    
    return Operator


In [25]:

def run_program(initial_state, program):
    '''
    Takes a list of dictionary elements, iterates through land finds quantum gates and target qubits, 
    we then pass in the gate and targets into get_operator()

    we then apply the operator to psi and then get a new psi to apply the next operator to and continue. 

    return th final state vector
    '''

    from math import log2 

    psi=initial_state
    total_qubits=int(log2(len(initial_state)))
    
    for i in range(len(program)): 

        gate_unitary=program[i]['gate']

        target_qubits=program[i]['target']

        O = get_operator(total_qubits, gate_unitary, target_qubits)

        psi=np.dot(psi,O)

    
    return psi

In [26]:

def measure_all(state_vector):
    '''
    
    creates a list of all possible states of system by calculating the total_qubits in system, the we map 
    the probablity amplitudes onto the possible states with a porbability distribution and use np.random.choice()
    to choose one according to our weights! 
    
    '''
    
    from math import log2 
    import numpy as np
    

    total_qubits=int(log2(len(state_vector)))
        
    # generate a list of possible states 
    
    possible_states=[]
    
    for i in range(len(state_vector)):
    
        string=bin(i)[2:][::-1] # helps get binary representation of states in our encoding of big endian 
                                #first splice gets rid of the '0b' in front of the string returned by bin
                                # second splice reverses the string 
        
        string+='0'*(total_qubits-len(string))
        
        possible_states.append(string)
    
    probability_distribution=(np.array(state_vector))**2 # probability is amplitude squared! 
    
    result=np.random.choice(possible_states,1, p=probability_distribution)
    
    return result[0]

In [27]:
def get_counts(state_vector, num_shots):
    
    '''
    Execute measure_all(), num_shots times and keep track of the results!! 
    '''

    counts={}
    
    for i in range(num_shots): 
        
        result=measure_all(state_vector)
        
        if result in counts: 
            counts[result]+=1
        else:
            counts[result]=1
    return counts

# Remarks 

I am extremely proud and excited to see how far I can go using numpy and some critical thinking. Being able to bridge my ability to do linear algebra on qubits on paper to being able to do any operation through some keystrokes!!! I did not think creating a simple QC compiler like this would have been so plausible. I wish I had more time to implement parametrized gates, but I believe I have the know-how and architecture to add more functionality in the future. It was fun going back to basics and just focusing on my coding. Sometimes in Qiskit for example, there are times where I can autopilot and put stuff in to run and not think as much about what I am doing, thinking of tensor products and the specific ordering of things, and how one would write these matrices out on paper. This was a rewarding experience that I hope to take with me to work on bigger projects in the future! 